In [1]:
import edward as ed
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import tensorflow as tf
import mobilenet_v1
CLASS_OF_INTEREST = 10
sess = tf.Session()

## Get some pretrained tensorflow network for our needs


$y = f(x)$, where y --- prediction, $f(\cdot)$ -- NN, x -- image(for example)

In [2]:
x = tf.Variable(tf.random_normal([224, 224, 3], stddev=0.35),name="input")
with tf.contrib.slim.arg_scope(mobilenet_v1.mobilenet_v1_arg_scope()):
    logits, net = mobilenet_v1.mobilenet_v1(tf.expand_dims(x, axis=0), is_training=False, num_classes=1001)
    
init_fn = tf.contrib.slim.assign_from_checkpoint_fn('./mobilenet_v1_1.0_224.ckpt', 
                                                        tf.contrib.slim.get_model_variables())
init_fn(sess)

INFO:tensorflow:Restoring parameters from ./mobilenet_v1_1.0_224.ckpt


## Construct from this network probability dension function for Edwardlib

1. We need density function

2. Some how need to define density from classifier output (wow so heuristic). I.e. if we want to sample images that belong to class 1:

$$p(x) = \frac{1}{Z} \left( f(x) - 1 + \epsilon\right)_+$$

Or smth more smooth.

In [3]:
from tensorflow.python.framework import dtypes
from edward.models import RandomVariable
from tensorflow.contrib.distributions import Distribution
from tensorflow.python.ops.distributions import distribution
EPS = 1e-20
CONST = 50
class NNRandomVariable(RandomVariable, Distribution):
    def __init__(self, *args, **kwargs):
        super(NNRandomVariable, self).__init__(*args, **kwargs)
        self.eps = 0.1
        
    def _log_prob(self, value):
        with tf.contrib.slim.arg_scope(mobilenet_v1.mobilenet_v1_arg_scope()):
            logits, net = mobilenet_v1.mobilenet_v1(tf.expand_dims(value, axis=0), reuse=True,
                                                    is_training=False, num_classes=1001)

        init_fn = tf.contrib.slim.assign_from_checkpoint_fn('./mobilenet_v1_1.0_224.ckpt', 
                                                                tf.contrib.slim.get_model_variables())
        init_fn(sess)
        classifier = tf.sigmoid(logits)[:, CLASS_OF_INTEREST]
        return -CONST * (classifier - 1 + self.eps)

    def _sample_n(self, n, seed=None):
        return tf.random_normal(dtype=dtypes.float32, shape=[n, 224, 224, 3])

In [4]:
NN = NNRandomVariable(dtype=dtypes.float32,
                      validate_args=False,
                      allow_nan_stats=True,
                      reparameterization_type=distribution.NOT_REPARAMETERIZED)

In [5]:
a = tf.Variable(tf.random_normal([224, 224, 3], stddev=0.35),name="a")
init = tf.global_variables_initializer()
init.run(session=sess)
res = tf.gradients(NN._log_prob(value=a), a)

INFO:tensorflow:Restoring parameters from ./mobilenet_v1_1.0_224.ckpt


In [6]:
NN = ed.models.Normal(loc=tf.ones([224, 224, 3]), scale=2.0 * tf.ones([224, 224, 3]))
empiric = ed.models.Empirical(params=tf.Variable(2. + tf.random_normal([500, 224, 224, 3])))
init = tf.global_variables_initializer()
init.run(session=sess)

In [7]:
with tf.name_scope("inference"):
    inference = ed.inferences.HMC({NN: empiric})
    inference.initialize()
    sess = ed.get_session()
    tf.global_variables_initializer().run()

    print(inference.n_iter)
    for i in range(inference.n_iter):
        info_dict = inference.update()
        inference.print_progress(info_dict)
        t = info_dict['t']
        if t % 100 == 0:
            print()
            pics = sess.run(empiric.get_variables())[0]
            print(pics.mean())
            print('Loss', sess.run(NN.log_prob(pics[1])).mean())
            print()

500
 10/500 [  2%]                                ETA: 88s | Acceptance Rate: 0.000 
1.9997
Loss -1.86162

 20/500 [  4%] █                              ETA: 53s | Acceptance Rate: 0.000
1.99962
Loss -1.86162

 30/500 [  6%] █                              ETA: 41s | Acceptance Rate: 0.000
1.99954
Loss -1.86162

 40/500 [  8%] ██                             ETA: 35s | Acceptance Rate: 0.000
1.99947
Loss -1.86162

 50/500 [ 10%] ███                            ETA: 31s | Acceptance Rate: 0.000
1.99941
Loss -1.86162

 60/500 [

In [8]:
sess.run(empiric.get_variables())[0]

array([[[[ 0.64244258,  0.95129108,  0.31718183],
         [ 1.59534585,  2.31182766,  1.9686718 ],
         [ 1.03903389,  0.80012167,  3.36637402],
         ..., 
         [ 3.20131755,  2.30033064, -0.11945844],
         [ 1.51400733,  2.95512009,  0.23125696],
         [ 2.4529705 ,  3.06067133,  2.29629016]],

        [[ 2.30872321,  2.78408146,  4.01946545],
         [ 1.94570696,  0.77270246,  1.17918098],
         [ 2.21998596,  3.08780885, -1.11257005],
         ..., 
         [ 0.19757986,  0.53924358,  1.1882987 ],
         [ 3.54100609,  1.08081138,  0.68205297],
         [ 2.11315465,  2.02942657,  0.50478816]],

        [[ 3.22460723,  2.05025673,  1.87231219],
         [ 0.11530101,  1.67652059,  1.71445334],
         [ 4.32448292,  2.1193316 , -1.06634498],
         ..., 
         [ 1.88726795,  3.01932049,  0.27497375],
         [ 2.95477009,  1.94157791,  1.66287732],
         [ 2.23451829,  1.56608987,  1.07996511]],

        ..., 
        [[ 2.14906311,  1.2876451 ,